# *23y 예약자 이용 행태 분석*

---
## 개요
1. 제주지역의 예약 패턴을 찾아본다

---

## 주요 명칭
1. Part
*   제주공항 : 제주공항 쏘카존
*   제주공항 앞 : 제주공항 앞(도보) 쏘카존
*   일반_제주 : 제주공항, 제주공항 앞을 제외한 제주지역 쏘카존
*   일반_서귀포 : 제주공항, 제주공항 앞을 제외한 서귀포지역 쏘카존

2. Leadtime
*   예약생성과 예약시작 시간간의 차이(Day)

3. 예약 생성위치
*   예약 당시 예약자의 위치(GPS)

4. 쏘카존
*   쏘카의 카셰어링이 운영되는 주차장 단위의 영업 거점

5. 쏘카존과의 거리
*   예약 생성위치와 쏘카존 간의 거리(M)

---


## 가설과 분석 방향 기획
*   *이용자의 예약 패턴에 따라 대여요금과 존 공급계획은 달라져야 한다*
*   *따라서 먼저 이용이 시작 되기전의 상황을 분석해 제주지역의 이용 트렌드를 분석해본다*
---

1. 문제 정의 : Part별로 서로 다른 이용 행태를 가지고 있고 그에 따라 전략 기획에 대한 접근 방법도 달라야 한다.


Part별 이용자들의 예약을 분석해본다
*   예약 리드타임
*   예약 생성시간
*   예약 생성위치

2. 기대효과
*   분류된 예약 패턴에 따라 사전 대응(대여요금 세팅, 존별 운영 계획 등)을 보다 효율적으로 진행할 수 있을 것으로 기대됨

---

---
## Summary

1. 제주도의 수요는 여행 수요가 대부분이다
2. 제주도 운영지역 Part별 이용 행태는 다르다
- 공항/공항 앞 : 제주도외 + 사전 + 당일 수요
- 일반 : 제주도내 + 당일 수요
3. 다른 행태에 따라 각 파트별 요금 전략과 공급계획의 중요한 기준은 달라져야한다

---

In [ ]:
from google.colab import output
output.clear()

# 필요한 라이브러리 준비 ⏳
---

In [ ]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import plotly.express as px

In [ ]:
import gspread
from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.cloud import bigquery
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(creds)

# 분석 파트 1 _ 예약 리드타임
---

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년, 제주지역 한정

In [ ]:
query = """
  WITH base1 AS (
    SELECT
    extract(year FROM date(start_at, 'Asia/Seoul')) as year,
    extract(month FROM date(start_at, 'Asia/Seoul')) as month,
    extract(isoweek FROM date(start_at, 'Asia/Seoul')) as week,
    r.id as rid,
    r.member_id as mid,

    CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront'
          WHEN zone_id IN (18471, 18472, 18473) THEN 'test' ELSE 'common' END as part,

    datetime_diff(end_at, start_at, hour) as dur,
    date(start_at, 'Asia/Seoul') as sdate,
    date(end_at, 'Asia/Seoul') as edate,
    extract(hour FROM r.created_at) as chour,
    extract(hour FROM start_at) as shour,
    datetime_diff(start_at, r.created_at, day) as leadtime,


  FROM `tianjin_replica.reservation_info` r LEFT JOIN `tianjin_replica.carzone_info` z ON r.zone_id = z.id
  WHERE r.member_imaginary IN (0, 9)
  AND date(start_at, 'Asia/Seoul') BETWEEN '2023-01-01' AND '2023-12-31'
  AND z.region1 = '제주특별자치도'
  AND r.state in (1,2,3)
  )

  SELECT
    year, month, week,
    part,
    leadtime,
    count(rid) as cnt
  FROM base1
  GROUP BY year, month, week, part, leadtime
  """
df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)


In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

## 📊 리드타임별 예약 건수 전체 대비 구성비
1. 당일 예약의 비중이 대다수이다.
2. 최대 10일이내의 예약이 많다

In [ ]:
# part와 leadtime별로 그룹화하고 cnt의 합계
summary = df.groupby('leadtime')['cnt'].sum().reset_index()

# 전체 예약건수의 합계
total_employee_counts = summary['cnt'].sum()

# 비율
summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100

fig = px.bar(summary, x='leadtime', y='proportion',
             title="각 leadtime별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'leadtime': 'Leadtime : Day'})


fig.update_traces(texttemplate='%{text:1.0f}%', textposition='outside')
fig.show()

In [ ]:
summary.head(11)

### ✈ 공항의 예약 리드타임 구성비
1. 공항 Part 역시 당일 예약의 비중이 높다
2. 공항은 30일 이상으로도 미리 예약하는 경우가 *있다*

In [ ]:
df_air = df[df['part']=='air']

summary = df_air.groupby(['part', 'leadtime'])['cnt'].sum().reset_index()

total_employee_counts = summary['cnt'].sum()

summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100


fig = px.bar(summary, x='leadtime', y='proportion', color='part',
             title="각 leadtime별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'leadtime': 'Leadtime', 'part': 'Part'})


fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
summary.head(11)

In [ ]:
# @title proportion

from matplotlib import pyplot as plt
summary['proportion'].plot(kind='hist', bins=20, title='proportion')
plt.gca().spines[['top', 'right',]].set_visible(False)

### ✈▶ 공항 앞 예약 리드타임 구성비
1. 공항 앞 역시 당일 예약의 비중이 대부분이다
2. 공항과 비슷한 패턴의 리드타임을 보여주지만 공항과 같이 리드타임이 길 수록 줄어들지 않는다

In [ ]:
df_airf = df[df['part']=='air_infront']


summary = df_airf.groupby(['part', 'leadtime'])['cnt'].sum().reset_index()

total_employee_counts = summary['cnt'].sum()

summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100

fig = px.bar(summary, x='leadtime', y='proportion', color='part',
             title="각 leadtime별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'leadtime': 'Leadtime', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
summary.head(11)

### 🏝 일반 지역의 예약 전체 구성비
1. 거의 모든 예약이 당일 예약이다
2. 5일 이내에 전체의 대부분의 예약이 분포되어 있다

In [ ]:
df_common = df[df['part']=='common']

summary = df_common.groupby(['part', 'leadtime'])['cnt'].sum().reset_index()

total_employee_counts = summary['cnt'].sum()

summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100

fig = px.bar(summary, x='leadtime', y='proportion', color='part',
             title="각 leadtime별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'leadtime': 'Leadtime', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
# @title
summary.head(11)

### ✅ 결론
1. 제주 전체적으로 당일 예약의 비중이 높다
2. 공항과 공항 앞은 사전에 미리 예약하는 비중도 꽤 높다
3. 제주 일반은 대체로 당일 예약하며 사전 3일을 넘어가는 경우는 거의 없다

In [ ]:
from google.colab import output
output.clear()

# 분석 파트 1 _ 예약 시작시간

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년, 제주지역 한정

---

In [ ]:
query = """
  WITH base1 AS (
    SELECT
    extract(year FROM date(start_at, 'Asia/Seoul')) as year,
    extract(month FROM date(start_at, 'Asia/Seoul')) as month,
    extract(isoweek FROM date(start_at, 'Asia/Seoul')) as week,
    r.id as rid,
    r.member_id as mid,

    CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront'
          WHEN zone_id IN (18471, 18472, 18473) THEN 'test' ELSE 'common' END as part,

    datetime_diff(end_at, start_at, hour) as dur,
    date(start_at, 'Asia/Seoul') as sdate,
    date(end_at, 'Asia/Seoul') as edate,
    extract(hour FROM r.created_at) as chour,
    extract(hour FROM start_at) as shour,
    datetime_diff(start_at, r.created_at, day) as leadtime,


  FROM `tianjin_replica.reservation_info` r LEFT JOIN `tianjin_replica.carzone_info` z ON r.zone_id = z.id
  WHERE r.member_imaginary IN (0, 9)
  AND date(start_at, 'Asia/Seoul') BETWEEN '2023-01-01' AND '2023-12-31'
  AND z.region1 = '제주특별자치도'
  AND r.state in (1,2,3)
  )

  SELECT
    year, month, week,
    part,
    shour,
    count(rid) as cnt
  FROM base1
  GROUP BY year, month, week, part, shour
  """
df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

## 📊 예약 시작시간별 예약 건수 전체 대비 구성비
1. 22시부터 13시 사이에 예약을 많이함
2. 당장 차를 사용하기 전날, 직전에 많이 예약하는 것으로 보여짐

In [ ]:
summary = df.groupby('shour')['cnt'].sum().reset_index()

# 전체 예약건수의 합계
total_employee_counts = summary['cnt'].sum()

# 비율
summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100

fig = px.bar(summary, x='shour', y='proportion',
             title="각 shour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'shour': 'shour : Hour'})

fig.update_traces(texttemplate='%{text:1.0f}%', textposition='outside')
fig.show()

In [ ]:
summary.shape

In [ ]:
summary.head(24)

### ✈ 공항의 예약 리드타임 구성비
1. 공항 Part는 제주 전체의 예약시간과 비슷한 트렌드를 보임
2. 늦은 저념부터 새벽시간대에 예약이 많이 되어짐

In [ ]:
df_air = df[df['part']=='air']

summary_air = df_air.groupby(['part', 'shour'])['cnt'].sum().reset_index()

total_employee_counts = summary_air['cnt'].sum()

summary_air['proportion'] = (summary_air['cnt'] / total_employee_counts) * 100

fig = px.bar(summary_air, x='shour', y='proportion', color='part',
             title="각 shour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'shour': 'Reservation Start hour', 'part': 'Part'})


fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
summary_air.shape

In [ ]:
summary_air.head(24)

### ✈▶ 공항 앞 예약 리드타임 구성비
1. 공항 앞 역시 전체 트렌드와 비슷함
2. 공항과 비슷한 패턴의 리드타임을 보여주지만 공항과 달리 특정 시간대에 예약을 하는 경우가 많음

In [ ]:
df_airf = df[df['part']=='air_infront']

summary_airf = df_airf.groupby(['part', 'shour'])['cnt'].sum().reset_index()

total_employee_counts = summary_airf['cnt'].sum()

summary_airf['proportion'] = (summary_airf['cnt'] / total_employee_counts) * 100

fig = px.bar(summary_airf, x='shour', y='proportion', color='part',
             title="각 shour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'shour': 'Reservation Start hour', 'part': 'Part'})


fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
summary_airf.shape

In [ ]:
summary_airf.head(24)

### 🏝 일반 지역의 예약 전체 구성비
1. 공항, 공항 앞 지역과는 다소 다르게 오후까지도 예약이 분포되어있고 저녁시간대 예약이 적음
2. 당장 차가 필요한 시간에 예약을 하는 것으로 보여짐

In [ ]:
df_common = df[df['part']=='common']

summary_common = df_common.groupby(['part', 'shour'])['cnt'].sum().reset_index()


total_employee_counts = summary_common['cnt'].sum()

summary_common['proportion'] = (summary_common['cnt'] / total_employee_counts) * 100

fig = px.bar(summary_common, x='shour', y='proportion', color='part',
             title="각 shour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'shour': 'Reservation Start hour', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
summary_common.shape

In [ ]:
summary_common.head(24)

### ✅ 결론
1. 대체로 늦은 저녁부터 오전에 이용을 시작하는 경우가 많다.
2. 공항과 공항 심야부터 오전까지로 집중되지만
3. 제주 일반은 오후까지 골고루 분포되는 트렌드를 보인다.
4. 늦은 오후부터 저녁시간 전까지는 예약이 대체로 없다 -> 해당 시간대에는 이동 수요가 없다 AND 이미 팔려 이용할 수 있는 차가 없다

# 분석 파트 1 _ 예약 생성시간

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년, 제주지역 한정

---

In [ ]:
query = """
  WITH base1 AS (
    SELECT
    extract(year FROM date(start_at, 'Asia/Seoul')) as year,
    extract(month FROM date(start_at, 'Asia/Seoul')) as month,
    extract(isoweek FROM date(start_at, 'Asia/Seoul')) as week,
    r.id as rid,
    r.member_id as mid,

    CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront'
          WHEN zone_id IN (18471, 18472, 18473) THEN 'test' ELSE 'common' END as part,

    datetime_diff(end_at, start_at, hour) as dur,
    date(start_at, 'Asia/Seoul') as sdate,
    date(end_at, 'Asia/Seoul') as edate,
    extract(hour FROM r.created_at) as chour,
    extract(hour FROM start_at) as shour,
    datetime_diff(start_at, r.created_at, day) as leadtime,


  FROM `tianjin_replica.reservation_info` r LEFT JOIN `tianjin_replica.carzone_info` z ON r.zone_id = z.id
  WHERE r.member_imaginary IN (0, 9)
  AND date(start_at, 'Asia/Seoul') BETWEEN '2023-01-01' AND '2023-12-31'
  AND z.region1 = '제주특별자치도'
  AND r.state in (1,2,3)
  )

  SELECT
    part,
    chour,
    count(rid) as cnt
  FROM base1
  GROUP BY part, chour
  """
df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

## 📊 예약 생성시간별 예약 건수 전체 대비 구성비
1. 예약이 시작하는 시간과 비슷한 트렌드를 보이지만, 생성은 15시 이전까지도 꾸준히 나타남
2. 예약시작도 비슷하지만 15시부터 22시이전까지는 생성과 시작이 거의 없는 시간 -> 이용할 차가 없기 때문일 수 있다

In [ ]:
summary = df.groupby('chour')['cnt'].sum().reset_index()

total_employee_counts = summary['cnt'].sum()

summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100

fig = px.bar(summary, x='chour', y='proportion',
             title="각 chour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'chour': '예약_생성시간'})

fig.update_traces(texttemplate='%{text:1.0f}%', textposition='outside')
fig.show()

In [ ]:
summary.shape

In [ ]:
summary.head(24)

### ✈ 공항의 예약시작 건수 구성비
1. 공항 Part는 제주 전체의 예약 생성시간과 비슷한 트렌드를 보임
2. 늦은 저녁부터 점심 이후까지 예약이 많이 생성됨

In [ ]:
df_air = df[df['part']=='air']

summary_air = df_air.groupby(['part', 'chour'])['cnt'].sum().reset_index()

total_employee_counts = summary_air['cnt'].sum()

summary_air['proportion'] = (summary_air['cnt'] / total_employee_counts) * 100

fig = px.bar(summary_air, x='chour', y='proportion', color='part',
             title="각 chour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'chour': '예약 생성시간', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
summary_air.shape

In [ ]:
summary_air.head(24)

### ✈▶ 공항 앞 예약 시작건수 구성비
1. 공항 앞 역시 전체 트렌드와 비슷하며 공항과 같은 구성을 보여주지만
2. 공항과는 다르게 특정 시간대에서 생성이 크게 증가 -> 비행기 탑승 전/후로 예약할 수 있다

In [ ]:
df_airf = df[df['part']=='air_infront']

summary_airf = df_airf.groupby(['part', 'chour'])['cnt'].sum().reset_index()

total_employee_counts = summary_airf['cnt'].sum()

summary_airf['proportion'] = (summary_airf['cnt'] / total_employee_counts) * 100

fig = px.bar(summary_airf, x='chour', y='proportion', color='part',
             title="각 chour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'chour': '예약 생성시간', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

In [ ]:
summary_air.shape

In [ ]:
summary_air.head(24)

### 🏝 일반 지역의 예약 전체 구성비
1. 전반적인 예약 생성의 트렌드는 공항과 비슷함
2. 오후 3시를 기점으로 예약 생성이 크게 줄어듦

In [ ]:
df_common = df[df['part']=='common']

# part와 leadtime별로 그룹화하고 cnt의 합계를 구합니다.
summary_common = df_common.groupby(['part', 'chour'])['cnt'].sum().reset_index()

# 전체 예약건수의 합계를 계산합니다.
total_employee_counts = summary_common['cnt'].sum()

# 비율을 계산합니다.
summary_common['proportion'] = (summary_common['cnt'] / total_employee_counts) * 100

# Plotly Express를 사용하여 그래프를 그립니다.
fig = px.bar(summary_common, x='chour', y='proportion', color='part',
             title="각 chour별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'chour': '예약 생성시간', 'part': 'Part'})

# texttemplate와 textposition을 조정합니다.
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')

# 그래프를 보여줍니다.
fig.show()

### ✅ 결론
1. 예약 시작과 생성은 대체로 비슷한 추이를 보인다
2. 공항과 일반 지역의 이용자들은 같다고 볼 수 있을 것이다 -> 생성과 시작의 패턴이 비슷하기 때문
3. 리드타임에서 보앗듯이 당장 필요해서 전날/당일 아침에 예약하는 경우가 많다
4. 공항 앞은 비행기의 착륙 전/후를 기준으로 이용이 많이 나타날 수 있다

---
### ⏩ 제주 지역의 예약 행태
1. 당일 수요가 대부분이다
- 공항과 공항 앞은 사전 수요도 있음
- 일반은 대부분 당일 수요

2. 이용 D-day 점심전후에 예약하는 경우가 대부분이다
- 판매는 오전에 대부분 이루어진다
- 판매를 위한 전날의 액션이, 다음날 판매에 영향을 준다

3. 파트별 행태
- 제주공항은 사전수요에 대한 전략, 심야~오전까지 공급피크 전략이 필요
- 제주공항 앞은 사전수요에 대한 전략, 비행기 편수에 따른 공급 전략 필요
- 제주일반은 당일 수요에 대응 하기 위해 D-1의 전략과 액션이 중요

### ↪ 예약 행태 분석의 한계
1. 시즌별, 요일별에 따른 구분이 이루어지지 않음
2. 전체 구성비를 가지고 보았기 때문에 운영대수가 많은 쏘카존의 영향도가 높을 수 있음

---

# 4. 참고 - 파트별 이용시간
- 예약시작 - 종료시간 = 이용시간

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년, 제주지역 한정

---

In [ ]:
query = """
  WITH base1 AS (
    SELECT
    extract(year FROM date(start_at, 'Asia/Seoul')) as year,
    extract(month FROM date(start_at, 'Asia/Seoul')) as month,
    extract(isoweek FROM date(start_at, 'Asia/Seoul')) as week,
    r.id as rid,
    r.member_id as mid,

    CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront'
          WHEN zone_id IN (18471, 18472, 18473) THEN 'test' ELSE 'common' END as part,

    datetime_diff(end_at, start_at, hour) as dur,
    date(start_at, 'Asia/Seoul') as sdate,
    date(end_at, 'Asia/Seoul') as edate,
    extract(hour FROM r.created_at) as chour,
    extract(hour FROM start_at) as shour,
    datetime_diff(start_at, r.created_at, day) as leadtime,


  FROM `tianjin_replica.reservation_info` r LEFT JOIN `tianjin_replica.carzone_info` z ON r.zone_id = z.id
  WHERE r.member_imaginary IN (0, 9)
  AND date(start_at, 'Asia/Seoul') BETWEEN '2023-01-01' AND '2023-12-31'
  AND z.region1 = '제주특별자치도'
  AND r.state in (1,2,3)
  )

  SELECT
    year, month, week,
    part,

    CASE WHEN dur <= 10 THEN 10
         WHEN dur > 10 AND dur <= 21 THEN 21
         WHEN dur > 21 AND dur <= 47 THEN 47
         WHEN dur > 47 AND dur <= 78 THEN 78 ELSE 79 END as dur,

    count(rid) as cnt
  FROM base1
  GROUP BY year, month, week, part, dur
  """

df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

## 📊 파트별 이용시간 구성비
1. 공항은 장시간(24시간 이상) 이용의 비율이 압도적이다
2. 공항 앞은 장시간(24시간 이상)과 단시간(10시간 이하)의 구성비 반반으로 나뉘어진다고 볼 수 잇다
3. 일반은 압도적으로 단시간(10시간 이하) 이용이 많다.

In [ ]:
# part와 leadtime별로 그룹화하고 cnt의 합계를 구합니다.
summary = df.groupby(['part', 'dur'])['cnt'].sum().reset_index()

# 전체 예약건수의 합계를 계산합니다.
total_employee_counts = summary['cnt'].sum()

# 비율을 계산합니다.
summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100

# Plotly Express를 사용하여 그래프를 그립니다.
fig = px.bar(summary, x='dur', y='proportion', color='part',
             title="각 dur별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'dur': '이용시간', 'part': 'Part'})

# texttemplate와 textposition을 조정합니다.
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')

# 그래프를 보여줍니다.
fig.show()

In [ ]:
summary = df.groupby(['part', 'dur'])['cnt'].sum().reset_index()

total_employee_counts = summary['cnt'].sum()

summary['proportion'] = (summary['cnt'] / total_employee_counts) * 100

# 각 'part'별 도넛차트
for part in summary['part'].unique():
    fig = px.pie(summary[summary['part'] == part], values='proportion', names='dur',
                 title=f"'{part}' Part의 dur 별 구성비",
                 hole=0.3)
    fig.update_traces(textinfo='percent+label')
    fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

summary = summary[summary['part'] != "test"]

# 각 'part'별로 subplot 생성을 위한 준비
parts = summary['part'].unique()
rows = 1  # 여기서는 모든 차트를 한 줄로 나열 (필요에 따라 조정)
cols = len(parts)  # unique 'part'의 개수만큼 열 생성

# make_subplots로 subplot 구조 생성
fig = make_subplots(rows=rows, cols=cols, specs=[[{'type':'domain'}]*cols], subplot_titles=[f"'{part}' Part" for part in parts])

# 각 'part'별 도넛 차트를 subplot에 추가
for i, part in enumerate(parts, start=1):
    part_summary = summary[summary['part'] == part]
    fig.add_trace(go.Pie(labels=part_summary['dur'], values=part_summary['proportion'], name=part, hole=0.3, textinfo='percent+label'), row=1, col=i)

# 차트 레이아웃 설정 (옵션)
fig.update_layout(title_text="각 'part'별 dur 별 구성비", width=1200, height=600)  # 너비와 높이는 적절하게 조절

# 차트 표시
fig.show()

# 분석 파트 2 _ 예약 생성위치
* 예약자의 예약 당시 위치를 기준으로 제주 지역과 존의 이용 행태를 분석해본다

---

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년, 제주지역 한정

---

# 시작하기에 앞서 _ 파트별 예약 위치 히트맵

In [ ]:
query = """
  WITH base AS (
    SELECT
      CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront' ELSE 'common' END as part,

      date,
      isoyear as year, isoweek as week, month,
      reservation_id as rid,
      reservation_created_lat as r_lat,
      reservation_created_lng as r_lng,
    FROM soda_store.reservation_v2
    WHERE date >= '2023-01-01' AND date <= '2023-12-31' AND region1 = '제주특별자치도'
  )

  SELECT
    part,
    rid,
    r_lat, r_lng
  FROM base
  """

df_map = pd.io.gbq.read_gbq(
  query=query,
  project_id="socar-data"
)

In [ ]:
df_map.info()

In [ ]:
df_map_air = df_map[df_map['part'] == 'air'].copy()
df_map_airf = df_map[df_map['part'] == 'air_infront'].copy()
df_map_common = df_map[df_map['part'] == 'common'].copy()

In [ ]:
df_map_air.info()

In [ ]:
!pip install folium --upgrade

In [ ]:
import folium
from folium.plugins import MarkerCluster

### ✈ 공항 예약위치 히트맵
- 공항은 데이터가 너무 많기 때문에 샘플링하여 노출

In [ ]:
df_map_air.dropna(subset=['r_lat', 'r_lng'], inplace=True)

df_sampled = df_map_air.sample(frac=0.3)

map_center = [df_sampled['r_lat'].mean(), df_sampled['r_lng'].mean()]

m = folium.Map(location=map_center, zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

for idx, row in df_sampled.iterrows():
    folium.Marker(location=[row['r_lat'], row['r_lng']], popup=str(row['rid'])).add_to(marker_cluster)

m

### ✈▶ 공항 앞 예약위치 히트맵

In [ ]:
# 데이터 전처리: 누락된 값을 제거
df_map_airf.dropna(subset=['r_lat', 'r_lng'], inplace=True)

# 지도의 중심을 데이터의 평균 위도와 경도로 설정
map_center = [df_map_airf['r_lat'].mean(), df_map_airf['r_lng'].mean()]

# Folium 지도 객체 생성
m = folium.Map(location=map_center, zoom_start=10)

# 마커 클러스터 객체 생성
marker_cluster = MarkerCluster().add_to(m)

# 데이터 포인트를 클러스터에 추가
for idx, row in df_map_airf.iterrows():
    folium.Marker(location=[row['r_lat'], row['r_lng']], popup=str(row['rid'])).add_to(marker_cluster)

# 지도 표시
m

### 🏝 일반 예약위치 히트맵

In [ ]:
# 데이터 전처리: 누락된 값을 제거
df_map_common.dropna(subset=['r_lat', 'r_lng'], inplace=True)

# 지도의 중심을 데이터의 평균 위도와 경도로 설정
map_center = [df_map_common['r_lat'].mean(), df_map_common['r_lng'].mean()]

# Folium 지도 객체 생성
m = folium.Map(location=map_center, zoom_start=10)

# 마커 클러스터 객체 생성
marker_cluster = MarkerCluster().add_to(m)

# 데이터 포인트를 클러스터에 추가
for idx, row in df_map_common.iterrows():
    folium.Marker(location=[row['r_lat'], row['r_lng']], popup=str(row['rid'])).add_to(marker_cluster)

# 지도 표시
m

# 분석 파트 2 _ 예약 생성 위치와 예약한 쏘카존과의 거리
- 파트별 이용자들은 어디서 예약할까 ?

In [ ]:
query = """
  WITH base AS (
    SELECT
      CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront' ELSE 'common' END as part,

      date,
      isoyear as year, isoweek as week, month,
      reservation_id as rid,
      zone_id as zid,
      zone_lng, zone_lat,
      age,
      reservation_created_lat as r_lat,
      reservation_created_lng as r_lng,

      CASE WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 500 THEN '500m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 1000 THEN '1000m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 3000 THEN '3000m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) > 3000 THEN 'O3000m' END as zone_loc,

      CASE WHEN st_distance(st_geogpoint(126.5319, 33.36266), st_geogpoint(reservation_created_lng, reservation_created_lat)) >= 50000 THEN 'jeju_out'
           WHEN st_distance(st_geogpoint(126.5319, 33.36266), st_geogpoint(reservation_created_lng, reservation_created_lat)) < 50000 THEN 'jeju_in' END as rev_loc

    FROM soda_store.reservation_v2
    WHERE date >= '2023-01-01' AND date <= '2023-12-31' AND region1 = '제주특별자치도'
  )

  SELECT
    part,
    zone_loc,
    count(rid) as cnt,
  FROM base
  WHERE zone_loc is not null
  GROUP BY part, zone_loc
  """
df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

df.info()

In [ ]:
df.shape

## 📊 파트별 예약 쏘카존과의 예약 위치 거리
1. 공항과 공항은 3km 이상에서 예약하는 건수가 압도적이다
2. 일반은 대체로 500m 이내에서 예약하는 건수가 많으며 공항 앞도 일부 예약은 근거리에서 예약되었다

In [ ]:
fig = px.bar(df, x='zone_loc', y='cnt', color='part',
             title="각 존예약위치별 예약건수",
             barmode='group',
             text='cnt',
             labels={'cnt': '이용건수 (건)', 'zone_loc': '예약위치_존', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

### ✈ 공항의 예약위치와 공항 쏘카존과의 거리
1. 공항은 82% 이상의 예약이 3km 이상에서 예약된다

* 공항은 쏘카존 주변이 아닌 더 먼 지역에서 예약된다

In [ ]:
df_air = df[df['part'] == 'air']
df_air = df_air.groupby(['part', 'zone_loc'])['cnt'].sum().reset_index()

total_employee_counts = df_air['cnt'].sum()

df_air['proportion'] = (df_air['cnt'] / total_employee_counts) * 100

fig = px.bar(df_air, x='zone_loc', y='proportion', color='part',
             title="각 zone_loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'zone_loc': '예약위치_존', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

### ✈▶ 공항 앞의 예약 위치와 공항 앞 쏘카존과의 거리
1. 공항 앞 역시 공항과 비슷하게 3km 이외 지역에서의 예약이 많다
2. 공항과 다른 점은 500m 이내에서 예약하는 건의 비율도 높다는 것(상대적으로)

In [ ]:
df_airf = df[df['part'] == 'air_infront']
df_airf = df_airf.groupby(['part', 'zone_loc'])['cnt'].sum().reset_index()

total_employee_counts = df_airf['cnt'].sum()

df_airf['proportion'] = (df_airf['cnt'] / total_employee_counts) * 100

fig = px.bar(df_airf, x='zone_loc', y='proportion', color='part',
             title="각 zone_loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'zone_loc': '예약위치_존', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

### 🏝 일반의 예약 위치와 일반지역내 쏘카존과의 거리
1. 일반은 대부분의 예약이 근거리에서 예약된다

In [ ]:
df_common = df[df['part'] == 'common']
df_common = df_common.groupby(['part', 'zone_loc'])['cnt'].sum().reset_index()

total_employee_counts = df_common['cnt'].sum()

df_common['proportion'] = (df_common['cnt'] / total_employee_counts) * 100

fig = px.bar(df_common, x='zone_loc', y='proportion', color='part',
             title="각 zone_loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'zone_loc': '예약위치_존', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

# 분석 파트 2 _ 제주도내/외의 예약 구성은 어떻게 될까?
- 제주도의 수요는 제주도내에서 만들어지는가, 외부에서 만들어지는가

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년, 제주지역 한정

---

In [ ]:
query = """
  WITH base AS (
    SELECT
      CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront' ELSE 'common' END as part,

      date,
      isoyear as year, isoweek as week, month,
      reservation_id as rid,
      zone_id as zid,
      zone_lng, zone_lat,
      age,
      reservation_created_lat as r_lat,
      reservation_created_lng as r_lng,

      CASE WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 500 THEN '500m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 1000 THEN '1000m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 3000 THEN '3000m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) > 3000 THEN 'O3000m' END as zone_loc,

      CASE WHEN st_distance(st_geogpoint(126.5319, 33.36266), st_geogpoint(reservation_created_lng, reservation_created_lat)) >= 50000 THEN 'jeju_out'
           WHEN st_distance(st_geogpoint(126.5319, 33.36266), st_geogpoint(reservation_created_lng, reservation_created_lat)) < 50000 THEN 'jeju_in' END as rev_loc

    FROM soda_store.reservation_v2
    WHERE date >= '2023-01-01' AND date <= '2023-12-31' AND region1 = '제주특별자치도'
  )

  SELECT
    part,
    rev_loc,
    count(rid) as cnt,
  FROM base
  WHERE rev_loc is not null
  GROUP BY part, rev_loc
  """

df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

df.info()

## 📊 파트별 도내외 예약
1. 공항과 공항 앞은 도외에서의 예약이 많다
2. 일반은 도내에서의 예약이 많다

In [ ]:
fig = px.bar(df, x='rev_loc', y='cnt', color='part',
             title="각 존예약위치별 예약건수",
             barmode='group',
             text='cnt',
             labels={'cnt': '이용건수 (건)', 'rev_loc': '예약위치_지역', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
# 각 `rev_loc`별로 데이터를 집계
summary = df.groupby(['rev_loc', 'part'])['cnt'].sum().reset_index()

# `rev_loc`의 unique 값들을 가져옵니다.
locations = summary['rev_loc'].unique()
cols = len(locations)  # unique 'rev_loc'의 개수만큼 열 생성

# make_subplots로 subplot 구조 생성
fig = make_subplots(rows=1, cols=cols, specs=[[{'type':'domain'}]*cols], subplot_titles=[f"'{loc}' 예약위치" for loc in locations])

# 각 `rev_loc`별 도넛 차트를 subplot에 추가
for i, loc in enumerate(locations, start=1):
    loc_summary = summary[summary['rev_loc'] == loc]
    total_cnt = loc_summary['cnt'].sum()
    loc_summary['proportion'] = (loc_summary['cnt'] / total_cnt) * 100

    fig.add_trace(go.Pie(labels=loc_summary['part'], values=loc_summary['proportion'], name=loc, hole=0.3, textinfo='percent+label'), row=1, col=i)

# 차트 레이아웃 설정 (옵션)
fig.update_layout(title_text="각 예약위치_지역별 Part 구성비", width=1000, height=400)  # 너비와 높이는 적절하게 조절

# 차트 표시
fig.show()

### ✈ 공항의 예약위치와 공항 쏘카존과의 거리
1. 공항은 도내보다 도외에서의 예약이 많다
2. 여행전 예약이 많음

In [ ]:
df_air = df[df['part'] == 'air']
df_air = df_air.groupby(['part', 'rev_loc'])['cnt'].sum().reset_index()

total_employee_counts = df_air['cnt'].sum()

df_air['proportion'] = (df_air['cnt'] / total_employee_counts) * 100

fig = px.bar(df_air, x='rev_loc', y='proportion', color='part',
             title="각 rev_loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'rev_loc': '예약위치_제주', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

### ✈▶ 공항 앞의 예약 위치와 공항 앞 쏘카존과의 거리
1. 공항 앞 역시 공항과 비슷한 추이를 보임

In [ ]:
df_airf = df[df['part'] == 'air_infront']
df_airf = df_airf.groupby(['part', 'rev_loc'])['cnt'].sum().reset_index()

total_employee_counts = df_airf['cnt'].sum()

df_airf['proportion'] = (df_airf['cnt'] / total_employee_counts) * 100

fig = px.bar(df_airf, x='rev_loc', y='proportion', color='part',
             title="각rev_loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'rev_loc': '예약위치_지역', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

### 🏝 일반의 예약 위치와 일반지역내 쏘카존과의 거리
1. 일반은 도내에서의 예약이 많다
2. 여행중 예약하는 경우가 많음

In [ ]:
df_common = df[df['part'] == 'common']
df_common = df_common.groupby(['part', 'rev_loc'])['cnt'].sum().reset_index()

total_employee_counts = df_common['cnt'].sum()

df_common['proportion'] = (df_common['cnt'] / total_employee_counts) * 100

fig = px.bar(df_common, x='rev_loc', y='proportion', color='part',
             title="각 rev_loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'rev_loc': '예약위치_지역', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

# 분석 파트 2 _ 예약 위치(거리/제주도내,외)에 따른 예약 구성비

### 필요한 데이터를 빅쿼리에 연동하여 가져옴
* 23년, 제주지역 한정

---

In [ ]:
query = """
  WITH base AS (
    SELECT
      CASE WHEN zone_id IN (105, 9890) THEN 'air'
          WHEN zone_id IN (17209) THEN 'air_infront' ELSE 'common' END as part,

      date,
      isoyear as year, isoweek as week, month,
      reservation_id as rid,
      zone_id as zid,
      zone_lng, zone_lat,
      age,
      reservation_created_lat as r_lat,
      reservation_created_lng as r_lng,

      CASE WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 500 THEN '500m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 1000 THEN '1000m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) <= 3000 THEN '3000m'
          WHEN st_distance(st_geogpoint(zone_lng, zone_lat), st_geogpoint(reservation_created_lng, reservation_created_lat)) > 3000 THEN 'O3000m' END as zone_loc,

      CASE WHEN st_distance(st_geogpoint(126.5319, 33.36266), st_geogpoint(reservation_created_lng, reservation_created_lat)) >= 50000 THEN 'jeju_out'
           WHEN st_distance(st_geogpoint(126.5319, 33.36266), st_geogpoint(reservation_created_lng, reservation_created_lat)) < 50000 THEN 'jeju_in' END as rev_loc

    FROM soda_store.reservation_v2
    WHERE date >= '2023-01-01' AND date <= '2023-12-31' AND region1 = '제주특별자치도'
  )

  SELECT
    part,
    zone_loc, rev_loc,
    count(rid) as cnt,
  FROM base
  WHERE zone_loc is not null AND rev_loc is not null
  GROUP BY part, rev_loc, zone_loc
  """

df = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)

df['loc'] = df['zone_loc'].fillna('') + df['rev_loc']

df.info()

## 📊 파트별 구성비
1. 공항과 공항 앞은 도외에서의 예약이 대부분인 것이 확실하다
2. 일반은 도내 근거리에서의 예약이 많으며, 도내의 원거리에서의 건수도 꽤 된다

In [ ]:
fig = px.bar(df, x='loc', y='cnt', color='part',
             title="각 존예약위치별 예약건수",
             barmode='group',
             text='cnt',
             labels={'cnt': '이용건수 (건)', 'loc': '예약위치_지역', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:1.0f}', textposition='outside')
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 각 `loc`별로 데이터를 집계
summary = df.groupby(['loc', 'part'])['cnt'].sum().reset_index()

# `loc`의 unique 값들을 가져옵니다.
locations = summary['loc'].unique()
cols = len(locations)  # unique 'loc'의 개수만큼 열 생성

# make_subplots로 subplot 구조 생성
fig = make_subplots(rows=1, cols=cols, specs=[[{'type':'domain'}] * cols], subplot_titles=[f"'{loc}' 예약위치" for loc in locations])

# 각 `loc`별 도넛 차트를 subplot에 추가
for i, loc in enumerate(locations, start=1):
    loc_summary = summary[summary['loc'] == loc]
    total_cnt = loc_summary['cnt'].sum()
    loc_summary['proportion'] = (loc_summary['cnt'] / total_cnt) * 100  # proportion 계산

    fig.add_trace(go.Pie(labels=loc_summary['part'], values=loc_summary['proportion'], name=loc, hole=0.3, textinfo='percent+label'), row=1, col=i)

# 차트 레이아웃 설정 (옵션)
fig.update_layout(title_text="각 예약위치_지역별 Part 구성비", width=1200, height=400)  # 너비와 높이는 적절하게 조절

# 차트 표시
fig.show()

### ✈ 공항 위치별 구성비

In [ ]:
df_air = df[df['part'] == 'air']
df_air = df_air.groupby(['part', 'loc'])['cnt'].sum().reset_index()

total_employee_counts = df_air['cnt'].sum()

df_air['proportion'] = (df_air['cnt'] / total_employee_counts) * 100

fig = px.bar(df_air, x='loc', y='proportion', color='part',
             title="각 loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'loc': '예약위치_거리', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

### ✈▶ 공항 앞 위치별 구성비

In [ ]:
df_airf = df[df['part'] == 'air_infront']
df_airf = df_airf.groupby(['part', 'loc'])['cnt'].sum().reset_index()

total_employee_counts = df_airf['cnt'].sum()

df_airf['proportion'] = (df_airf['cnt'] / total_employee_counts) * 100

fig = px.bar(df_airf, x='loc', y='proportion', color='part',
             title="각 loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'loc': '예약위치_거리', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

### 🏝 일반 예약위치별 구성비

In [ ]:
df_common = df[df['part'] == 'common']
df_common = df_common.groupby(['part', 'loc'])['cnt'].sum().reset_index()

total_employee_counts = df_common['cnt'].sum()

df_common['proportion'] = (df_common['cnt'] / total_employee_counts) * 100

fig = px.bar(df_common, x='loc', y='proportion', color='part',
             title="각 loc별 예약건수의 전체 대비 구성비",
             barmode='group',
             text='proportion',
             labels={'proportion': '구성비 (%)', 'loc': '예약위치_거리', 'part': 'Part'})

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

### ✅ 결론
1. 공항과 공항 앞의 예약은 대부분 제주도외에서 예약된다
2. 일반은 대부분 근거리에서 예약된다

## 분석 결과

*제주도의 예약은 파트에 따라 예약 성향이 나뉘어진다*
- 공항,공항 앞 : 당일수요 + 사전수요 + 제주도외
- 일반 : 당일수요 + 제주도내 근거리

*지역별 대여요금 체계는 달라야하며, 공급 기획시 사전 수요와 도내 유동인구의 영향을 고려해야한다*
- 공항, 공항 앞 = 사전수요
- 일반 = 당일수요

## 분석의 한계점
- 쏘카존을 파트별로 묶어 보았기 때문에 운영대수가 많은 존의 영향도가 커져 어뷰징이 있을 수 있음
- 연간 예약 전체에 대해 분석하였기 때문에 시즌에 대한 요인은 반영되지 않음

추가해야할거 : 예약위치 -> 존별 폴리엄맵 위에서 예약위치 히트맵 보여주기

In [ ]:
query = """
  SELECT
    id as zid,
    CASE WHEN id IN (105,9890) THEN '공항'
        WHEN id IN (17209, 19175) THEN '공항앞'
        ELSE '일반'
        END as part,
    lng, lat
  FROM `tianjin_replica.carzone_info`
  WHERE region1 = '제주특별자치도'
  AND state = 1
  """

df_zone = pd.io.gbq.read_gbq(
    query=query,
    project_id="socar-data"
)


df_zone.info()

In [ ]:
df_zone['part'].unique()

In [ ]:
# part별 색상 지정
color_dict = {'일반': 'blue', '공항': 'red', '공항앞': 'green'}

# 기본 지도 생성, 중심 좌표와 줌 레벨을 설정
map = folium.Map(location=[37.497950, 127.027587], zoom_start=10)

# 데이터프레임의 각 행에 대해 반복
for index, row in df_zone.iterrows():
    # 마커 생성 및 지도에 추가
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=f"zid: {row['zid']}",
        icon=folium.Icon(color=color_dict[row['part']])
    ).add_to(map)

# 지도 표시
map